In [ ]:
import pandas as pd
from neuralforecast import NeuralForecast

In [ ]:
# Load and preprocess your dataset
Y_df = pd.read_csv('../data/average_weekly_rating_messenger.csv')

# Ensure the 'ds' column is datetime
Y_df['ds'] = pd.to_datetime(Y_df.iloc[:, 0])

# Ensure the 'y' column is numeric and handle any non-numeric issues
Y_df['y'] = pd.to_numeric(Y_df.iloc[:, 1], errors='coerce')
Y_df.dropna(subset=['y'], inplace=True)  # Drop rows where 'y' is NaN after conversion

# Add unique_id as a constant value
Y_df['unique_id'] = 'rating'

# Keep only necessary columns
Y_df = Y_df[['unique_id', 'ds', 'y']]

# Check the DataFrame to confirm structure and types
print(Y_df.dtypes)  # 'unique_id' should be object, 'ds' datetime64, 'y' float64
print(Y_df.head())

In [ ]:
# Define validation and test sizes
n_time = len(Y_df['ds'].unique())
val_size = int(0.2 * n_time)
test_size = int(0.2 * n_time)

In [ ]:
from neuralforecast.core import NeuralForecast
from neuralforecast.models import Autoformer, PatchTST

In [ ]:
horizon = 48  # Forecasting 1 years

models = [
    Autoformer(h=horizon,
               input_size=horizon,
               max_steps=200,
               val_check_steps=50,
               early_stop_patience_steps=3),
    PatchTST(h=horizon,
             input_size=horizon,
             max_steps=200,
             val_check_steps=50,
             early_stop_patience_steps=3),
]

In [ ]:
nf = NeuralForecast(
    models=models,
    freq='W'  # Weekly frequency for your dataset
)

Y_hat_df = nf.cross_validation(
    df=Y_df,
    val_size=val_size,
    test_size=test_size,
    n_windows=None
)

In [ ]:
Y_hat_df.head()

In [ ]:
import matplotlib.pyplot as plt

# Get the latest cutoff for filtering predictions
latest_cutoff = Y_hat_df['cutoff'].max()
Y_hat_filtered = Y_hat_df[Y_hat_df['cutoff'] == latest_cutoff]

# Plot the results (both historical data and filtered future predictions)
fig, ax = plt.subplots(figsize=(10, 6))

# Plot historical data
Y_df.set_index('ds')['y'].plot(ax=ax, label='Historical Data', color='blue', linewidth=2)

# Plot filtered future predictions from the latest cutoff in cross-validation results
Y_hat_filtered.set_index('ds')['Autoformer'].plot(ax=ax, label='Autoformer Predictions', color='orange', linewidth=2)

# Predict zone
prediction_start_date = Y_hat_filtered['ds'].min() 
plt.axvline(x=prediction_start_date, color='red', linestyle='--', linewidth=1, label='Prediction Start')

# Formatting the plot
ax.set_title('Messenger Rating Predictions Using Autoformer Model', fontsize=16)
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('Rating', fontsize=14)

ax.legend(loc='upper right')
ax.grid()

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Get the latest cutoff for filtering predictions
latest_cutoff = Y_hat_df['cutoff'].max()
Y_hat_filtered = Y_hat_df[Y_hat_df['cutoff'] == latest_cutoff]

# Plot the results (both historical data and filtered future predictions)
fig, ax = plt.subplots(figsize=(10, 6))

# Plot historical data
Y_df.set_index('ds')['y'].plot(ax=ax, label='Historical Data', color='blue', linewidth=2)

# Plot filtered future predictions from the latest cutoff in cross-validation results
Y_hat_filtered.set_index('ds')['PatchTST'].plot(ax=ax, label='PatchTST Predictions', color='green', linewidth=2)

# Predict zone
prediction_start_date = Y_hat_filtered['ds'].min() 
plt.axvline(x=prediction_start_date, color='red', linestyle='--', linewidth=1, label='Prediction Start')

# Formatting the plot
ax.set_title('Messenger Rating Predictions Using PatchTST Model', fontsize=16)
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('Rating', fontsize=14)

ax.legend(loc='upper right')
ax.grid()

plt.tight_layout()
plt.show()